In [2]:
import pandas as pd
import math
from decimal import *
import numpy as np
from geopy.distance import vincenty

In [60]:
dfAden = pd.read_csv('./data/aden.csv',sep=';')

# Affectation des zones

In [61]:
min_lat=-10
max_lat=31
min_lon=31
max_lon=78

#un degre de latitude "vaut"  111,11 km
accuracy_km=65
accuracy_lat=accuracy_km/111.11

diff_lat=max_lat-min_lat
diff_lon=max_lon-min_lon

path_lat=float(diff_lat)/float(accuracy_lat)

latitude_grid=np.linspace(min_lat, max_lat, path_lat)
longitude_grid=[]
longitude_grid.append(min_lon)

tab=[]
for i in range(0,len(latitude_grid)-1):
    mean_lat=float(latitude_grid[i]+latitude_grid[i+1])/2
    
    #print mean_lat
    #print math.cos(math.radians(mean_lat))
    km_lon=111.11*math.cos(math.radians(mean_lat))
    accuracy_lon=accuracy_km/km_lon
    
    tab.append(accuracy_lon)
    
    currentLon=longitude_grid[i]+accuracy_lon
    longitude_grid.append(currentLon)
    #print 111.11*math.cos(math.radians(mean_lat))

    
#on complete jusqu'a 78
maxCurrent=max(longitude_grid)
meanPath=np.mean(tab)
while maxCurrent<78:
    maxCurrent=max(longitude_grid)
    longitude_grid.append(maxCurrent+meanPath)
    
    
print len(latitude_grid)
print len(longitude_grid)
#print latitude_grid
print longitude_grid


70
80
[31, 31.593495841258633, 32.185972950854925, 32.77749833108579, 33.36813845760308, 33.95795931801276, 34.547026449809984, 35.13540497770493, 35.72315965039237, 36.31035487681666, 36.89705476198269, 37.48332314236187, 38.069223620941685, 38.654819601965976, 39.24017432541272, 39.825350901255234, 40.410412343552274, 40.99542160441203, 41.58044160787476, 42.16553528375859, 42.75076560151286, 43.336195604123574, 43.92188844211536, 44.50790740769495, 45.094315969081194, 45.681177805067264, 46.26855683986128, 46.85651727825219, 47.44512364114857, 48.034440801539, 48.62453402092363, 49.21546898626784, 49.807311847529995, 50.40012925581715, 50.99398840222365, 51.58895705740979, 52.1851036119793, 52.7824971177166, 53.38120732974716, 53.981304749686416, 54.582860669845715, 55.185947218566305, 55.79063740675562, 56.39700517570336, 57.00512544625837, 57.61507416945126, 58.226928378651685, 58.84076624335363, 59.456667124686824, 60.07471163275748, 60.694981685926805, 61.31756057214194, 61.9425

In [62]:
def affectation_zone(latitude,longitude,latitude_grid,longitude_grid,lat_bool):
    #print 'latitude : '+ str(latitude)
    #print 'longitude : '+ str(longitude)
    
    for i in range(0,len(latitude_grid)-1):
        if latitude<latitude_grid[i+1] and latitude>latitude_grid[i]:
            idx_zone_lat=i
            
    for i in range(0,len(longitude_grid)-1):
        if longitude<longitude_grid[i+1] and longitude>longitude_grid[i]:
            idx_zone_lon=i

    #print idx_zone_lon
    #print idx_zone_lat
    if lat_bool:
        return idx_zone_lat
    else:
        return idx_zone_lon

In [63]:
latitude=9.91666666667
longitude=76.0833333333
idx_zone_lat=affectation_zone(latitude,longitude,latitude_grid,longitude_grid,True)
idx_zone_lon=affectation_zone(latitude,longitude,latitude_grid,longitude_grid,False)
print idx_zone_lat
print idx_zone_lon

33
74


In [64]:
dfAden['lat_area'] = dfAden.apply(lambda row: affectation_zone(row['latitude'],row['longitude'],latitude_grid,longitude_grid,True), axis=1)
dfAden['lon_area'] = dfAden.apply(lambda row: affectation_zone(row['latitude'],row['longitude'],latitude_grid,longitude_grid,False), axis=1)

# Affectation des quinzines

In [72]:
def quinzaine(month,day):
    if day<15:
        return ((month-1))*2+1
    else:
        return ((month-1)*2)+2
quinzaine(2.0,23.0)

4.0

In [74]:
dfAden['quinzaine_week']=dfAden.apply(lambda row: quinzaine(row['mois (0)'],row['jour (0)']), axis=1)

In [75]:
dfAden.to_csv("./data/aden.csv",sep=';', encoding='utf-8')